In [1]:
import gurobipy as gp
import math
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter, PercentFormatter
import numpy as np
import pandas as pd
import re
import os
import shutil

In [2]:
# output generation for paper 2

In [3]:
# get input paths
test_set = "bm23"
instance_fldr = os.path.join("instances", test_set)
test_set_fldr = os.path.join("test_sets", test_set)
results_fldr = os.path.join("results", test_set)
out_fldr = os.path.join("outputs", test_set)

# set filters
seed_idxs = [0]  
max_indices = 11
degrees = [0, 2, 4]
term_list = [4, 64]
filter_cbc = False
max_base_std = 1e10
min_termination_time = 1
short, medium, long = 60, 600, 3600
remove_status_changes = False
win_threshold = .1

generators = ["None", "New", "Farkas", "All", "Disjunction", "Matrix", "Term", "Basis", "NoDisjunction", "NoMatrix", "NoTerm", "NoBasis"]

# set up some mappings
cat_map_new_lines = {
    "None": "Default",
    "Farkas": "Param Disj,\nParam Cuts",
    "Old": "Param Disj,\nCalc Cuts",
    "New": "Calc Disj,\nCalc Cuts"
}
cat_map = {
    "None": "Default",
    "Farkas": "Param Disj, Param Cuts",
    "Old": "Param Disj, Calc Cuts",
    "New": "Calc Disj, Calc Cuts"
}
perturbation_map = {
    "matrix": "Coefficient Matrix",
    "rhs": "Right Hand Side",
    "bounds": "Variable Bounds",
    "objective": "Objective"
}
label = {
    "postRootTime": "Time after Processing Root nodes",
    "rootDualBoundTimeSansVpc": "Root Processing Time (Minus VPC Generation)",
    "terminationTimeSansVpc": "Time (Minus VPC Generation)",
    "terminationTime": "Time",
    "nodes": "Nodes Processed",
    "iterations": "LP iterations",
}
unit = {
    "postRootTime": "(seconds)",
    "rootDualBoundTimeSansVpc": "(seconds)",
    "terminationTimeSansVpc": "(seconds)",
    "terminationTime": "(seconds)",
    "nodes": "(1000 nodes)",
    "iterations": "(1000 iterations)",
}
limits = {
    "postRootTime": 7200,
    "terminationTimeSansVpc": 7200,
    "terminationTime": 7200,
    "rootDualBoundTimeSansVpc": 5,
    "nodes": 10000,
    "iterations": 37500
}
bracket_bounds = {
    "short": (min_termination_time, short),
    "medium": (short, medium),
    "long": (medium, long)
}
param_map = {
    "degree": "Degree of Perturbation",
    "terms": "Number of Disjunctive Terms",
}

In [4]:
# matplotlib settings
plt.rc('text', usetex=True)  # use latex fonts
plt.rcParams['font.size'] = 18
plt.rcParams['figure.titlesize'] = 24
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 14

## Check run failures

In [5]:
# check if each folder in test_set_fldr has a corresponding .mps file in instance_fldr
# for instance in os.listdir(test_set_fldr):
#     if not os.path.isdir(os.path.join(test_set_fldr, instance)):
#         continue
#     if not os.path.exists(os.path.join(instance_fldr, f"{instance}.mps")):
#         # remove the folder if the instance is missing
#         # shutil.rmtree(os.path.join(test_set_fldr, instance))
#         print(f"Removed {instance} from test set")

In [6]:
# running list of strings contained by different error codes
# last two are catchalls
err = {
    "walltime": [],
    "bad_alloc": [],
    "out of memory": [],
    "takeoffcuts": [],
    "solver is dual infeasible": [],
    "solver must be optimal": [],
    "segmentation fault": [],
    "no vpcs were made from a new disjunction": [],
    "must have primalbound >= root lp objective": [],
    "objective at parent nodes": [],
    "failed to optimize mip": [],
    "disjunction does not represent a full binary tree": [],
    "solver not proven optimal for nodes": [],
    "unable to open": [],
    "license": [],
    "dot product with obj differs from solver": [],
    "gurobi: error during callback: addCut": [],
    "cglvpc::setupconstraints: objective at disjunctive term": [],
    "unable to read file": [],
    "stats.id == stats_vec": [],
    "size of our disjunction is not what we expected it to be": [],
    "dimension must stay fixed": [],
    "vpcgenerator must be": [],
}

# read in cbc acceptable instances from cbc.txt
with open("cbc.txt", "r") as f:
    cbc_instances = f.read().split("\n")

# runs that errored out with new error code
other = []

# runs that had no errors
empty = []

# runs that only had warnings
warn_strs = ["warning", "prlp is primal infeasible", "farkas", "x:", "x[", "b:",
             "b[", "v:", "v[", "cut:", "A_i . x", "dot product with obj differs from solver"]
warning = []

# series that didn't run
no_go = []

# track sizes of instances
rows, cols, density = {}, {}, {}

# map the names
names = {}

# counts
count_series = 0
count_instances = 0
number_instances = {}

# iterate over all expected runs
for instance in os.listdir(test_set_fldr):
    if not os.path.isdir(os.path.join(test_set_fldr, instance)):
        continue
    # only look at cbc instances if we ran with cbc
    if instance not in cbc_instances and "gurobi" not in test_set and filter_cbc:
        continue
        
    # get the number of rows and columns in the instance
    mdl = gp.read(os.path.join(instance_fldr, f"{instance}.mps"))
    rows[instance] = mdl.NumConstrs
    cols[instance] = mdl.NumVars
    density[instance] = mdl.NumNZs / (mdl.NumConstrs * mdl.NumVars)
        
    for perturbation in os.listdir(os.path.join(test_set_fldr, instance)):
        if not os.path.isdir(os.path.join(test_set_fldr, instance, perturbation)):
            continue
        # only look at perturbations that were run
        p, d = perturbation.split("_")
        if int(d) not in degrees:
            continue
        for terms in term_list:
            for generator in generators:
                for seed_idx in seed_idxs:

                    # set variables for this iterations
                    count_series += 1
                    stem = f"{instance}_{perturbation}_{terms}_{generator}_{seed_idx}"
                    file_pth = os.path.join(results_fldr, f"{stem}.err")
                    series_fldr = os.path.join(test_set_fldr, instance, perturbation)
                    current_count = len([f for f in os.listdir(series_fldr) if f.endswith(".mps")])
                    count_instances += current_count
                    names[stem] = instance
                    number_instances[stem] = {
                        "expected": current_count,
                        "recorded": 0,
                        "generator": generator,
                        "error": "N/A"
                    }
    
                    # check if the series wasn't run
                    if not os.path.exists(file_pth):
                        number_instances[stem]["error"] = "no go"
                        no_go.append(stem)
                    
                    # check if the series ran with no errors or warnings
                    elif os.path.getsize(file_pth) == 0:
                        number_instances[stem]["error"] = "empty"
                        empty.append(stem)
                    
                    # track which error codes were thrown
                    else:
                        # read the file
                        with open(file_pth, "r") as f:
                            text = f.read().lower()
                        
                        # assign the error file to the appropriate list
                        found_code = False
                        for code in err:
                            if code in text:
                                if code == "dot product with obj differs from solver":
                                    pattern = r"obj viol from solver: (-?\d+\.\d+)\. calculated: (-?\d+\.\d+)"
                                    s, c = re.findall(pattern, text)[-1]
                                    # if we didn't terminate, this isn't an error, so keep going
                                    if abs(float(s) - float(c)) < 1e-3:
                                        continue
                                err[code].append(stem)
                                found_code = True
                                number_instances[stem]["error"] = code
                                break
                        if not found_code:
                            if all(not line or any(w in line for w in warn_strs) for line in text.splitlines()):
                                warning.append(stem)
                                number_instances[stem]["error"] = "warning"
                            else:
                                other.append(stem)
                                number_instances[stem]["error"] = "other"

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Read MPS format model from file instances/bm23/bm23.mps
Reading time = 0.00 seconds
BM23: 20 rows, 27 columns, 478 nonzeros


In [7]:
# check which series didn't run
print(no_go)

[]


In [8]:
# get the proportion of series that at least got started
1 - (len(no_go) / count_series)

1.0

In [9]:
# out of time - got hung up in code somewhere - ok
print(err["walltime"])
len(err["walltime"]) / count_series

[]


0.0

In [10]:
# out of memory - memory is maxed already - this is what it is
# todo: figure out where we ran short on memory so we can explain why we dropped them
print(err["bad_alloc"] + err["out of memory"])
len(err["bad_alloc"] + err["out of memory"]) / count_series

[]


0.0

In [11]:
# rerun this if want to give more memory to some instances
# bad_alloc_names = set(n.split("_")[0] for n in err["bad_alloc"])
# mem = pd.read_csv("more_memory.csv", index_col=0)
# mem["reason"] = "hard solve" 
# 
# for n in bad_alloc_names:
#     if f"{n}.mps" not in mem.index:
#         new_row = pd.DataFrame([{'file_name': f"{n}.mps", 'memory': 16.0, 'reason': 'big disjunction'}]).set_index('file_name')
#         mem = pd.concat([mem, new_row])
#     else:
#         mem.loc[f'{n}.mps', 'memory'] = 16.0
# 
# mem.to_csv("more_memory.csv")

In [12]:
# this is an issue with John's bookkeeping - not much we can do here
print(err["takeoffcuts"])
len(err["takeoffcuts"]) / count_series

[]


0.0

In [13]:
print(err["solver is dual infeasible"])
len(err["solver is dual infeasible"]) / count_series

[]


0.0

In [14]:
# these are usually issues with CLP finding optimality - not much we can do here
print(err["solver must be optimal"])
len(err["solver must be optimal"]) / count_series

[]


0.0

In [15]:
print(err["segmentation fault"])
len(err["segmentation fault"]) / count_series

[]


0.0

In [16]:
# seg_err = {
#     "Bad image at line": [],
# }
# 
# seg_other = []
# 
# for stem in err["segmentation fault"]:
#     file_pth = os.path.join(results_fldr, f"{stem}.out")
# 
#     with open(file_pth, "r") as f:
#         text = f.read()
#     
#     # assign the error file to the appropriate list
#     found_code = False
#     for code in seg_err:
#         if code in text:
#             seg_err[code].append(stem)
#             found_code = True
#             break
#     if not found_code:
#         seg_other.append(stem)

In [17]:
# print(seg_err["Bad image at line"])
# len(seg_err["Bad image at line"]) / len(err["segmentation fault"]) if err["segmentation fault"] else 0

In [18]:
# print(seg_other)
# len(seg_other)/len(err["segmentation fault"]) if err["segmentation fault"] else 0

In [19]:
# # get breakdown of why vpc generation failed - mostly from lack of provisioning
# for code, exps in seg_err.items():
#     print(f"{code}: {len(exps) / len(err['segmentation fault']) if err['segmentation fault'] else 0}")
# 
# print(f"other: {len(seg_other) / len(err['segmentation fault']) if err['segmentation fault'] else 0}")

In [20]:
# todo: check aleks' removals and drop those below for similar reasons
# todo: check size of disjunctions and decide what to do with those that are too big
# these should all be from the problem being too big and hitting the time limit or integer solutions
print(err["no vpcs were made from a new disjunction"])
missing_4_term = [n for n in err["no vpcs were made from a new disjunction"] if "_4_" in n]
missing_64_term = [n for n in err["no vpcs were made from a new disjunction"] if "_64_" in n]
print(f'4 term: {len(missing_4_term) / count_series}')
print(f'64 term: {len(missing_64_term) / count_series}')

[]
4 term: 0.0
64 term: 0.0


In [21]:
# vpc_err = {
#     "CglVPC: Finishing with exit reason: PRLP_TIME_LIMIT": [],
#     "CglVPC: Finishing with exit reason: TIME_LIMIT": [],
#     "CglVPC: Finishing with exit reason: NO_CUTS_LIKELY": [],
#     "CglVPC: Finishing with exit reason: PRLP_INFEASIBLE": [],
#     "CglVPC: Finishing with exit reason: SUCCESS": [],
#     "CglVPC: Finishing with exit reason: OPTIMAL_SOLUTION_FOUND": [],
#     "CglVPC: Finishing with exit reason: FAIL_LIMIT": [],
#     "CglVPC: Finishing with exit reason: NO_DISJUNCTION": [],
# }
# 
# vpc_other = []
# 
# for stem in err["no vpcs were made from a new disjunction"]:
#     file_pth = os.path.join(results_fldr, f"{stem}.out")
# 
#     with open(file_pth, "r") as f:
#         text = f.read()
#     
#     # assign the error file to the appropriate list
#     found_code = False
#     for code in vpc_err:
#         if code in text:
#             vpc_err[code].append(stem)
#             found_code = True
#             break
#     if not found_code:
#         vpc_other.append(stem)

In [22]:
# print(vpc_err["CglVPC: Finishing with exit reason: PRLP_TIME_LIMIT"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: PRLP_TIME_LIMIT"]) / len(err["no vpcs were made from a new disjunction"])

In [23]:
# print(vpc_err["CglVPC: Finishing with exit reason: TIME_LIMIT"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: TIME_LIMIT"]) / len(err["no vpcs were made from a new disjunction"])

In [24]:
# print(vpc_err["CglVPC: Finishing with exit reason: NO_CUTS_LIKELY"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: NO_CUTS_LIKELY"]) / len(err["no vpcs were made from a new disjunction"])

In [25]:
# print(vpc_err["CglVPC: Finishing with exit reason: PRLP_INFEASIBLE"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: PRLP_INFEASIBLE"]) / len(err["no vpcs were made from a new disjunction"])

In [26]:
# print(vpc_err["CglVPC: Finishing with exit reason: SUCCESS"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: SUCCESS"]) / len(err["no vpcs were made from a new disjunction"])

In [27]:
# print(vpc_err["CglVPC: Finishing with exit reason: OPTIMAL_SOLUTION_FOUND"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: OPTIMAL_SOLUTION_FOUND"]) / len(err["no vpcs were made from a new disjunction"])

In [28]:
# print(vpc_err["CglVPC: Finishing with exit reason: FAIL_LIMIT"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: FAIL_LIMIT"]) / len(err["no vpcs were made from a new disjunction"])

In [29]:
# print(vpc_err["CglVPC: Finishing with exit reason: NO_DISJUNCTION"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: NO_DISJUNCTION"]) / len(err["no vpcs were made from a new disjunction"])

In [30]:
# vpc_other

In [31]:
# # get breakdown of why vpc generation failed - mostly from lack of provisioning/problem being too large
# if err["no vpcs were made from a new disjunction"]:
#     for code, exps in vpc_err.items():
#         print(f"{code}: {len(exps) / len(err['no vpcs were made from a new disjunction'])}")
#     
#     print(f"other: {len(vpc_other) / len(err['no vpcs were made from a new disjunction'])}")

In [32]:
print(err["must have primalbound >= root lp objective"])
len(err["must have primalbound >= root lp objective"]) / count_series

[]


0.0

In [33]:
# LP relaxation objective is not going to match root nodes objective when warm starting 
print(err["objective at parent nodes"])
len(err["objective at parent nodes"]) / count_series

[]


0.0

In [34]:
# not enough tolerance added to bound (or we hit time limit) - element 2 from 5 and 4 from 4
print(err["failed to optimize mip"])
len(err["failed to optimize mip"]) / count_series

[]


0.0

In [35]:
# todo: figure out why
print(err["disjunction does not represent a full binary tree"])
len(err["disjunction does not represent a full binary tree"]) / count_series

['bm23_matrix_4_64_New_0']


0.004629629629629629

In [36]:
# again issue with not getting through vpc generation in time
# todo: handle this gracefully
print(err["solver not proven optimal for nodes"])
len(err["solver not proven optimal for nodes"]) / count_series

[]


0.0

In [37]:
print(err["unable to open"])
len(err["unable to open"]) / count_series

[]


0.0

In [38]:
print(err["license"])
len(err["license"]) / count_series

[]


0.0

In [39]:
print(warning)
len(warning) / count_series

['bm23_rhs_0_64_New_0', 'bm23_rhs_4_64_New_0', 'bm23_rhs_2_64_New_0', 'bm23_objective_2_64_New_0', 'bm23_objective_4_64_New_0']


0.023148148148148147

In [40]:
# errors unaccounted for
print(other)
len(other) / count_series

['bm23_objective_0_64_NoDisjunction_0', 'bm23_rhs_4_4_NoBasis_0', 'bm23_matrix_2_64_New_0']


0.013888888888888888

In [41]:
# proportion of series that were improperly provisioned
(len(err["bad_alloc"] + err["out of memory"] + err["walltime"])) / count_series

0.0

In [42]:
# todo handle this
print(err["dot product with obj differs from solver"])
len(err["dot product with obj differs from solver"]) / count_series

[]


0.0

In [43]:
# changed code to ignore this error
print(err["gurobi: error during callback: addCut"])
len(err["gurobi: error during callback: addCut"]) / count_series

[]


0.0

In [44]:
# largely not replicating - only issue I could find was aleks missing updated objective from CLP when resolving to check this
print(err["cglvpc::setupconstraints: objective at disjunctive term"])
len(err["cglvpc::setupconstraints: objective at disjunctive term"]) / count_series

[]


0.0

In [45]:
# not replicating - rerun
print(err["unable to read file"])
len(err["unable to read file"]) / count_series

[]


0.0

In [46]:
# not replicating - rerun
print(err["stats.id == stats_vec"])
len(err["stats.id == stats_vec"]) / count_series

[]


0.0

In [47]:
print(err["size of our disjunction is not what we expected it to be"])
len(err["size of our disjunction is not what we expected it to be"]) / count_series

[]


0.0

In [48]:
print(err["vpcgenerator must be"])
len(err["vpcgenerator must be"]) / count_series

[]


0.0

In [49]:
print(err["dimension must stay fixed"])
len(err["dimension must stay fixed"]) / count_series

[]


0.0

In [50]:
# get breakdown of errors
for code, exps in err.items():
    print(f"{code}: {len(exps) / count_series}")

print(f"other: {len(other) / count_series}")

print(f"warning: {len(warning) / count_series}")

print(f"no errors/warnings: {len(empty) / count_series}")

print(f"no go: {len(no_go) / count_series}")

walltime: 0.0
bad_alloc: 0.0
out of memory: 0.0
takeoffcuts: 0.0
solver is dual infeasible: 0.0
solver must be optimal: 0.0
segmentation fault: 0.0
no vpcs were made from a new disjunction: 0.0
must have primalbound >= root lp objective: 0.0
objective at parent nodes: 0.0
failed to optimize mip: 0.0
disjunction does not represent a full binary tree: 0.004629629629629629
solver not proven optimal for nodes: 0.0
unable to open: 0.0
license: 0.0
dot product with obj differs from solver: 0.0
gurobi: error during callback: addCut: 0.0
cglvpc::setupconstraints: objective at disjunctive term: 0.0
unable to read file: 0.0
stats.id == stats_vec: 0.0
size of our disjunction is not what we expected it to be: 0.0
dimension must stay fixed: 0.0
vpcgenerator must be: 0.0
other: 0.013888888888888888
no errors/warnings: 0.9583333333333334
no go: 0.0


## Read in data

In [51]:
# map generator names to the corresponding data frames
df_map = {g: pd.DataFrame() for g in generators} 
gap_map = {g: pd.DataFrame() for g in generators}
regex = re.compile(r'([a-zA-Z0-9-]+(?:_o)?)_([a-z]+)_([0-9-]+)_([0-9]+)_([a-zA-Z ]+)')
solution_pattern = r"_(\d+)\.pb"

# declaring types as needed
column_types = {
    "lpBound": float,
    "lpBoundPostVpc": float,
    "disjunctiveDualBound": float,
    "primalBound": float,
    "rootDualBound": float,
    "dualBound": float
}

skipped_instances = set()
primal_bounds = {}
same_solution = {}

# iterate over all files in the folder
for file_name in os.listdir(results_fldr):
    
    file_pth = os.path.join(results_fldr, file_name)
    
    # if the file is not a nonempty csv, skip it
    if not file_name.endswith(".csv") or os.path.getsize(file_pth) == 0:
        continue
    
    # get the experimental set up
    match = regex.search(file_name)
    instance_name = names.get(file_name[:-4])
    if not instance_name:
        skipped_instances.add(file_name[:-4].split("_")[0])
        os.remove(file_pth)
        continue
    # instance_name = match.group(1)
    perturbation = match.group(2)
    assert perturbation in ["matrix", "rhs", "bounds", "objective"], f"Unknown perturbation: {perturbation}"
    expo = int(match.group(3))
    assert expo in degrees, f"Unknown degree: {expo}"
    degree = 2**int(expo)
    terms = int(match.group(4))
    assert terms in term_list, f"Unknown number of terms: {terms}"
    generator = match.group(5)
    assert generator in generators, f"Unknown generator: {generator}"
    base_name = f"{instance_name}_0"
    
    # get the primal bounds for this experiment
    cur_instance_test_set_fldr = os.path.join(test_set_fldr, instance_name, f"{perturbation}_{expo}")
    for test_set_file in os.listdir(cur_instance_test_set_fldr):
        if test_set_file.endswith(".pb"):
            with open(os.path.join(cur_instance_test_set_fldr, test_set_file), "r") as f:
                primal_bounds[perturbation, expo, ".".join(test_set_file.split(".")[:-1])] = float(f.read())
                
    # see if solution changed
    for test_set_file in os.listdir(cur_instance_test_set_fldr):
        if test_set_file.endswith(".pb"):
            perturbation_name = ".".join(test_set_file.split(".")[:-1])
            same_solution[perturbation, expo, perturbation_name] = \
                primal_bounds[perturbation, expo, base_name] == primal_bounds[perturbation, expo, perturbation_name]
            
    # read the file
    df = pd.read_csv(file_pth, keep_default_na=False, dtype=column_types, index_col=0)
    
    for instance_idx in df.index:
        
        # fill in primal bounds if missing
        # df.loc[instance_idx, "primalBound"] = min(primal_bounds.get(stem_map.get(instance_idx), 1e100), df.loc[instance_idx, "primalBound"])
        df.loc[instance_idx, "primalBound"] = min(
            primal_bounds[perturbation, expo, f"{instance_name}_{instance_idx}"], df.loc[instance_idx, "primalBound"]
        )
        
        # same with root dual bound
        df.loc[instance_idx, "rootDualBound"] = df.loc[instance_idx, "rootDualBound"] if df.loc[instance_idx, "rootDualBound"] < 1e100 else df.loc[instance_idx, "lpBoundPostVpc"] 
    
    # get rid of the index so the rest of the notebook works
    df.reset_index(inplace=True)
    
    # add some identifying columns
    df["instance"] = instance_name
    df["perturbation"] = perturbation
    df["degree"] = degree
    df["terms"] = terms
    df["rows"] = rows[instance_name]
    df["cols"] = cols[instance_name]
    df["density"] = density[instance_name]
    
    # append to the appropriate data frame
    df_map[generator] = pd.concat([df_map[generator], df])
    
    # track recorded vs expected experiments
    number_instances[file_name[:-4]]["recorded"] = len(df)

In [52]:
# convert number_instances to dataframe
frame = pd.DataFrame(number_instances).T
frame.head()

,expected,recorded,generator,error
bm23_rhs_0_4_None_0,21,21,None,empty
bm23_rhs_0_4_New_0,21,21,New,empty
bm23_rhs_0_4_Farkas_0,21,21,Farkas,empty
bm23_rhs_0_4_All_0,21,21,All,empty
bm23_rhs_0_4_Disjunction_0,21,21,Disjunction,empty


In [53]:
# redo the runs that have incomplete data that we're not sure should be that way
redos = frame.loc[(frame["expected"] > frame["recorded"]) & (frame["error"] != "no vpcs were made from a new disjunction")].index.tolist()
redos = pd.DataFrame({"experiment": redos})
redos.to_csv("redos.csv", index=False)

In [54]:
if "miplib" in test_set or "quick" in test_set:
    # group frame by generator and sum remaining columns
    gb = frame.groupby(["generator", "error"]).sum().reset_index()
    gb["missing"] = gb["expected"] - gb["recorded"]
    total = gb.groupby("generator")[["expected", "missing"]].sum().reset_index()
    gb = pd.merge(gb, total, on="generator", suffixes=("", " total"))
    gb["ratio missing (by generator)"] = gb["missing"] / gb["missing total"]
    gb["ratio missing (by generator)"] = gb["ratio missing (by generator)"].apply(lambda x: round(x, 4))
    gb = gb.loc[:, ~gb.columns.str.contains("total")]  # get rid of the total columns
    gb.set_index(["generator", "error"], inplace=True)
    gb.to_csv(os.path.join(out_fldr, "missing_table.csv"), index=False, mode="w")
else:
    gb = None
gb

In [55]:
for gen in generators:
    masks = {
        0: -1e20 > df_map[gen]["lpBound"],
        1: df_map[gen]["lpBound"] - 1e-3 > df_map[gen]["lpBoundPostVpc"],
        2: (df_map[gen]["lpBoundPostVpc"] - 1e-3 > df_map[gen]["disjunctiveDualBound"]) & ((gen == "None") | (gen == "New")),
        3: df_map[gen]["rootDualBound"] - 1e-3 > df_map[gen]["dualBound"],
        4: (df_map[gen]["dualBound"] - 1e-3 > df_map[gen]["primalBound"]) & (df_map[gen]["dualBound"] / df_map[gen]["primalBound"] > 1 + 1e-3),
        5: df_map[gen]["primalBound"] > 1e20,
        6: 0 > df_map[gen]["vpcGenerationTime"],
        7: df_map[gen]["vpcGenerationTime"] - 1e-3 > df_map[gen]["rootDualBoundTime"],
        8: df_map[gen]["rootDualBoundTime"] - 1e-3 > df_map[gen]["terminationTime"],
        9: df_map[gen]["vpcGenerationTime"] - 1e-3 > df_map[gen]["bestSolutionTime"],
        10: df_map[gen]["bestSolutionTime"] - 1e-3 > df_map[gen]["terminationTime"]
    }
    for i, mask in masks.items():
        print(f"{gen} {i}: {mask.sum() / len(df_map[gen])}")

None 0: 0.0
None 1: 0.0
None 2: 0.0
None 3: 0.0
None 4: 0.0
None 5: 0.0
None 6: 0.0
None 7: 0.0
None 8: 0.0
None 9: 0.0
None 10: 0.0
New 0: 0.0
New 1: 0.0
New 2: 0.0
New 3: 0.0
New 4: 0.0
New 5: 0.0
New 6: 0.0
New 7: 0.0
New 8: 0.0
New 9: 0.0
New 10: 0.0
Farkas 0: 0.0
Farkas 1: 0.0
Farkas 2: 0.0
Farkas 3: 0.0
Farkas 4: 0.0
Farkas 5: 0.0
Farkas 6: 0.0
Farkas 7: 0.0
Farkas 8: 0.0
Farkas 9: 0.0
Farkas 10: 0.0
All 0: 0.0
All 1: 0.0
All 2: 0.0
All 3: 0.0
All 4: 0.0
All 5: 0.0
All 6: 0.0
All 7: 0.0
All 8: 0.0
All 9: 0.0
All 10: 0.0
Disjunction 0: 0.0
Disjunction 1: 0.0
Disjunction 2: 0.0
Disjunction 3: 0.0
Disjunction 4: 0.0
Disjunction 5: 0.0
Disjunction 6: 0.0
Disjunction 7: 0.0
Disjunction 8: 0.0
Disjunction 9: 0.0
Disjunction 10: 0.0
Matrix 0: 0.0
Matrix 1: 0.0
Matrix 2: 0.0
Matrix 3: 0.0
Matrix 4: 0.0
Matrix 5: 0.0
Matrix 6: 0.0
Matrix 7: 0.0
Matrix 8: 0.0
Matrix 9: 0.0
Matrix 10: 0.0
Term 0: 0.0
Term 1: 0.0
Term 2: 0.0
Term 3: 0.0
Term 4: 0.0
Term 5: 0.0
Term 6: 0.0
Term 7: 0.0
Term 8:

In [56]:
# it shouldn't be possible that dual bound > primal bound. this only happens when we use the saved primal bound, which was used to set the dual bound
df_map["Farkas"][masks[0]]

,instanceIndex,seedIndex,vpcGenerator,terms,lpBound,disjunctiveDualBound,lpBoundPostVpc,rootDualBound,dualBound,primalBound,...,tighten_disjunction,tighten_matrix_perturbation,tighten_infeasible_to_feasible_term,tighten_feasible_to_infeasible_basis,instance,perturbation,degree,rows,cols,density


In [57]:
for gen in df_map:
    mask = (-1e20 > df_map[gen]["lpBound"]) | \
        (df_map[gen]["lpBound"] - 1e-3 > df_map[gen]["lpBoundPostVpc"]) | \
        ((df_map[gen]["lpBoundPostVpc"] - 1e-3 > df_map[gen]["disjunctiveDualBound"]) & (gen != "Farkas")) | \
        (df_map[gen]["rootDualBound"] - 1e-3 > df_map[gen]["dualBound"]) | \
        ((df_map[gen]["dualBound"] - 1e-3 > df_map[gen]["primalBound"]) & (df_map[gen]["dualBound"] / df_map[gen]["primalBound"] > 1 + 1e-3)) | \
        (df_map[gen]["primalBound"] > 1e20) | \
        (0 > df_map[gen]["vpcGenerationTime"]) | \
        (df_map[gen]["vpcGenerationTime"] - 1e-3 > df_map[gen]["rootDualBoundTime"]) | \
        (df_map[gen]["rootDualBoundTime"] - 1e-3 > df_map[gen]["terminationTime"]) | \
        (df_map[gen]["vpcGenerationTime"] - 1e-3 > df_map[gen]["bestSolutionTime"]) | \
        (df_map[gen]["bestSolutionTime"] - 1e-3 > df_map[gen]["terminationTime"])
    print(f"{gen}: {mask.sum() / len(df_map[gen])}")
    df_map[gen] = df_map[gen][~mask]

None: 0.0
New: 0.0
Farkas: 0.0
All: 0.0
Disjunction: 0.0
Matrix: 0.0
Term: 0.0
Basis: 0.0
NoDisjunction: 0.0
NoMatrix: 0.0
NoTerm: 0.0
NoBasis: 0.0


In [58]:
# merge the different data frames into one
join_cols = ["instance", "perturbation", "degree", "terms", "instanceIndex", "seedIndex"]
df = df_map[generators[0]].merge(df_map[generators[1]], on=join_cols, suffixes=(f" {generators[0]}", None))
for g1, g2 in zip(generators[1:-1], generators[2:]):
    df = df.merge(df_map[g2], on=join_cols, suffixes=(f" {g1}", None if g2 != generators[-1] else f" {g2}"))
df.head()

,instanceIndex,seedIndex,vpcGenerator None,terms,lpBound None,disjunctiveDualBound None,lpBoundPostVpc None,rootDualBound None,dualBound None,primalBound None,...,termRemainsFeasibleBasisInfeasible NoBasis,cutsChangedCoefficients NoBasis,feasibleTermsPrunedByBound NoBasis,tighten_disjunction NoBasis,tighten_matrix_perturbation NoBasis,tighten_infeasible_to_feasible_term NoBasis,tighten_feasible_to_infeasible_basis NoBasis,rows NoBasis,cols NoBasis,density NoBasis
0,0,0,None,64,20.570922,20.570922,20.570922,26.427088,34.0,34.0,...,0,0,0,0,0,0,0,20,27,0.885185
1,10,0,None,64,20.661706,20.661706,20.661706,25.938525,34.0,34.0,...,4,0,34,1,1,1,0,20,27,0.885185
2,11,0,None,64,20.570922,20.570922,20.570922,26.427088,34.0,34.0,...,7,0,32,1,1,1,0,20,27,0.885185
3,12,0,None,64,20.077726,20.077726,20.077726,25.242914,33.0,33.0,...,6,0,30,1,1,1,0,20,27,0.885185
4,15,0,None,64,20.570922,20.570922,20.570922,26.427088,34.0,34.0,...,0,0,32,1,1,1,0,20,27,0.885185


In [59]:
# get proportion of tests run to completion
len(generators) * len(df) / count_instances

0.8783068783068783

In [60]:
def gap_closed(df, col):
    gap = abs(df[col] - df["lpBound None"]) / abs(df['primalBound None'] - df["lpBound None"])
    gap[(gap > 1) | (gap == np.nan)] = 1  # get corner cases
    return gap

# Function to map values based on a dictionary
def check_same_solution(row):
    # Create a tuple of the key based on the key_columns
    return same_solution[row["perturbation"], int(math.log2(row["degree"])), f'{row["instance"]}_{row["instanceIndex"]}']

In [61]:
# find the optimality gap closed by each generator
df["Disjunction (New)"] = gap_closed(df, "disjunctiveDualBound New")
df["Disjunction (Old)"] = gap_closed(df, "disjunctiveDualBound Farkas")
for g in generators:
    if g != "None":
        df[f"VPCs ({g})"] = gap_closed(df, f"lpBoundPostVpc {g}")        
    df[f"Root Cuts ({g})"] = gap_closed(df, f"rootDualBound {g}")

df["Root Optimality Gap Improvement"] = df["Root Cuts (Farkas)"] - df["Root Cuts (None)"] 
# df = df.dropna()

In [62]:
# find times without vpc generation
df["terminationTimeSansVpc None"] = df["terminationTime None"]
df["rootDualBoundTimeSansVpc None"] = df["rootDualBoundTime None"]
for gen in generators:
    if gen != "None":
        df[f"terminationTimeSansVpc {gen}"] = df[f"terminationTime {gen}"] - df["vpcGenerationTime New"]
        df[f"rootDualBoundTimeSansVpc {gen}"] = df[f"rootDualBoundTime {gen}"] - df[f"vpcGenerationTime {gen}"]
    df[f"postRootTime {gen}"] = df[f"terminationTime {gen}"] - df[f"rootDualBoundTime {gen}"]
    if gen not in ["None", "New"]:
        df[f"terminationTimeImprovement {gen}"] = (df["terminationTime None"] - df[f"terminationTime {gen}"]) / df["terminationTime None"]
        df[f"terminationTimeSansVpcImprovement {gen}"] = (df["terminationTimeSansVpc None"] - df[f"terminationTimeSansVpc {gen}"]) / df["terminationTimeSansVpc None"]
        df[f"nodesImprovement {gen}"] = (df["nodes None"] - df[f"nodes {gen}"]) / df["nodes None"] 
        df[f"iterationsImprovement {gen}"] = (df["iterations None"] - df[f"iterations {gen}"]) / df["iterations None"] 
        df[f"terminationTimeRatio {gen}"] = df[f"terminationTime {gen}"] / df["terminationTime None"]
        df[f"terminationTimeSansVpcRatio {gen}"] = df[f"terminationTimeSansVpc {gen}"] / df["terminationTimeSansVpc None"]
        df[f"nodesRatio {gen}"] = df[f"nodes {gen}"] / df["nodes None"] 
        df[f"iterationsRatio {gen}"] = df[f"iterations {gen}"] / df["iterations None"]
        df[f"nodesImproves {gen}"] = df["nodes None"] > df[f"nodes {gen}"]
        df[f"terminationTimeImproves {gen}"] = df["terminationTime None"] > df[f"terminationTime {gen}"]
        df[f"terminationTimeSansVpcImproves {gen}"] = df["terminationTimeSansVpc None"] > df[f"terminationTimeSansVpc {gen}"]
        df[f"iterationsImproves {gen}"] = df["iterations None"] > df[f"iterations {gen}"]
        df[f'nodesWin{gen}'] = df['nodes None']*(1 - win_threshold) > df[f'nodes {gen}']
        df[f'terminationTimeWin{gen}'] = df['terminationTime None']*(1 - win_threshold) > df[f'terminationTime {gen}']
        df[f'terminationTimeSansVpcWin{gen}'] = df['terminationTimeSansVpc None']*(1 - win_threshold) > df[f'terminationTimeSansVpc {gen}']
        df[f'iterationsWin{gen}'] = df['iterations None']*(1 - win_threshold) > df[f'iterations {gen}']
        df[f'nodesWinNoneVs{gen}'] = df[f'nodes {gen}']*(1 - win_threshold) > df['nodes None']
        df[f'terminationTimeWinNoneVs{gen}'] = df[f'terminationTime {gen}']*(1 - win_threshold) > df['terminationTime None']
        df[f'terminationTimeSansVpcWinNoneVs{gen}'] = df[f'terminationTimeSansVpc {gen}']*(1 - win_threshold) > df['terminationTimeSansVpc None']
        df[f'iterationsWinNoneVs{gen}'] = df[f'iterations {gen}']*(1 - win_threshold) > df['iterations None']
df["bracket"] = ["short" if t <= short else "medium" if t <= medium else "long" for t in df["terminationTime None"]]
df["sameSolution"] = df.apply(check_same_solution, axis=1)

/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_23200/3168479867.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"nodesRatio {gen}"] = df[f"nodes {gen}"] / df["nodes None"]
/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_23200/3168479867.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"iterationsRatio {gen}"] = df[f"iterations {gen}"] / df["iterations None"]
/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_23200/3168479867.py:18: PerformanceWarning: DataFrame is highly fragmented

In [63]:
# get sensitivity stats as ratios
for gen_name in generators:
    if gen_name == "None":
        continue
    df[f"infeasibleTermsRatio {gen_name}"] = df[f"infeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]
    df[f"infeasibleToFeasibleTermsRatio {gen_name}"] = df[f"infeasibleToFeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]
    df[f"zeroInfeasibleToFeasibleTerms {gen_name}"] = df[f"infeasibleToFeasibleTerms {gen_name}"] == 0
    df[f"feasibleToInfeasibleTermsRatio {gen_name}"] = df[f"feasibleToInfeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]

/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_23200/393337708.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"infeasibleTermsRatio {gen_name}"] = df[f"infeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]
/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_23200/393337708.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"infeasibleToFeasibleTermsRatio {gen_name}"] = df[f"infeasibleToFeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]
/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn

In [64]:
def optimality_gap(df, generator=None):
    if generator:
        return abs(df[f"primalBound {generator}"] - df[f"dualBound {generator}"]) / \
            abs(df[f"primalBound {generator}"])
    else:
        return abs(df[f"primalBound"] - df[f"dualBound"]) / abs(df[f"primalBound"])

In [65]:
# aleks filters
# df = df.loc[df["terms"] == df["actualTerms Farkas"]]
# df = df.loc[df["zeroInfeasibleToFeasibleTerms Farkas"]]

In [66]:
df.head()

,instanceIndex,seedIndex,vpcGenerator None,terms,lpBound None,disjunctiveDualBound None,lpBoundPostVpc None,rootDualBound None,dualBound None,primalBound None,...,zeroInfeasibleToFeasibleTerms NoMatrix,feasibleToInfeasibleTermsRatio NoMatrix,infeasibleTermsRatio NoTerm,infeasibleToFeasibleTermsRatio NoTerm,zeroInfeasibleToFeasibleTerms NoTerm,feasibleToInfeasibleTermsRatio NoTerm,infeasibleTermsRatio NoBasis,infeasibleToFeasibleTermsRatio NoBasis,zeroInfeasibleToFeasibleTerms NoBasis,feasibleToInfeasibleTermsRatio NoBasis
0,0,0,None,64,20.570922,20.570922,20.570922,26.427088,34.0,34.0,...,True,0.000000,0.031250,0.0,True,0.000000,0.031250,0.0,True,0.000000
1,10,0,None,64,20.661706,20.661706,20.661706,25.938525,34.0,34.0,...,True,0.015625,0.046875,0.0,True,0.015625,0.046875,0.0,True,0.015625
2,11,0,None,64,20.570922,20.570922,20.570922,26.427088,34.0,34.0,...,True,0.000000,0.031250,0.0,True,0.000000,0.031250,0.0,True,0.000000
3,12,0,None,64,20.077726,20.077726,20.077726,25.242914,33.0,33.0,...,True,0.000000,0.031250,0.0,True,0.000000,0.031250,0.0,True,0.000000
4,15,0,None,64,20.570922,20.570922,20.570922,26.427088,34.0,34.0,...,True,0.000000,0.031250,0.0,True,0.000000,0.031250,0.0,True,0.000000


In [67]:
# set aside core columns and filter for all subsequent dataframes
group_cols = ["instance", "perturbation", "bracket", "degree", "terms"]
id_cols = ["instanceIndex"]

# keep the instance, perturbation, instanceIndex triples that exist for all combinations of degree and terms
# where VPC did not find the optimal solution
full_df = df.loc[df["Disjunction (New)"] < .9999]
triples = (full_df.groupby(
        ["instance", "perturbation", "instanceIndex"]
    ).size().reset_index().rename(columns={0: "count"}))
triples.head()

,instance,perturbation,instanceIndex,count
0,bm23,matrix,0,6
1,bm23,matrix,1,6
2,bm23,matrix,2,4
3,bm23,matrix,3,5
4,bm23,matrix,4,5


In [68]:
# uncomment to filter for only the triples that exist for all combinations of degree and terms (and seed index)
# triples = triples[triples["count"] == len(degrees) * len(term_list) * len(seed_idxs)]
# full_df = full_df.merge(triples, on=["instance", "perturbation", "instanceIndex"])
full_df.to_csv(os.path.join(out_fldr, "cleaned_combined_complete.csv"), index=False, mode="w")

## Check Root Node Stats

In [69]:
def interleave(list_of_lists):
    return [item for sublist in zip(*list_of_lists) for item in sublist]

In [70]:
# additional filtering for dataframe on bounds
fields = ["Disjunction (New)", "Disjunction (Old)"] + [f"VPCs ({gen_name})" for gen_name in generators if gen_name != "None"] + \
    interleave([[f"Root Cuts ({gen_name})", f"terminationTime {gen_name}", f"nodes {gen_name}",
                 f"iterations {gen_name}", f"terminationTimeSansVpc {gen_name}", f"vpcGenerationTime {gen_name}", 
                 f"rootDualBoundTime {gen_name}"]
                for gen_name in generators]) + \
    interleave([[f"infeasibleTermsRatio {gen_name}", f"infeasibleToFeasibleTermsRatio {gen_name}",
                 f"zeroInfeasibleToFeasibleTerms {gen_name}", f"feasibleToInfeasibleTermsRatio {gen_name}"]
                for gen_name in generators if gen_name != "None"])

# now reduce bound_df to just the perturbed instances - make > -1 to include base instance
bound_df = full_df.loc[full_df["instanceIndex"] > 0, group_cols + id_cols + fields]

In [71]:
def geometric_mean(series, offset=1e-6):
    adjusted_series = series + offset  # Add a small offset to avoid zeros
    return np.exp(np.log(adjusted_series).mean())

# paper currently uses mean, but we can switch to geometric mean if we want
aggregations = {f: geometric_mean if f not in ["sameSolution"] else "mean" for f in fields}
aggregations["instance"] = "nunique"
aggregations["instanceIndex"] = "count"

In [72]:
# get gap closed by degree and term
out = bound_df.groupby(["degree", "terms"]).agg(aggregations).reset_index()
out.to_csv(os.path.join(out_fldr, "bound_table.csv"), index=False, mode="w")
out

/Users/sean/miniconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,degree,terms,Disjunction (New),Disjunction (Old),VPCs (New),VPCs (Farkas),VPCs (All),VPCs (Disjunction),VPCs (Matrix),VPCs (Term),...,feasibleToInfeasibleTermsRatio Disjunction,feasibleToInfeasibleTermsRatio Matrix,feasibleToInfeasibleTermsRatio Term,feasibleToInfeasibleTermsRatio Basis,feasibleToInfeasibleTermsRatio NoDisjunction,feasibleToInfeasibleTermsRatio NoMatrix,feasibleToInfeasibleTermsRatio NoTerm,feasibleToInfeasibleTermsRatio NoBasis,instance,instanceIndex
0,1,4,0.140507,0.148274,0.140292,0.144259,0.144310,0.144259,0.144310,0.144259,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1,60
1,1,64,0.679605,0.707561,0.663024,0.494280,0.663293,0.656976,0.501961,0.642353,...,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,1,51
2,4,4,0.111304,0.120178,0.108766,0.078547,0.086747,0.078547,0.085841,0.078547,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1,60
3,4,64,0.663510,0.626379,0.649138,0.017885,0.527745,0.149929,0.242688,0.133191,...,0.000028,0.000028,0.000028,0.000028,0.000028,0.000028,0.000028,0.000028,1,40
4,16,4,0.114902,0.010146,0.104119,0.000039,0.000065,0.000039,0.000065,0.000039,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1,60
5,16,64,0.545098,0.283714,0.524853,0.000205,0.000638,0.000205,0.000827,0.000205,...,0.000693,0.000693,0.000693,0.000693,0.000693,0.000693,0.000693,0.000693,1,43


In [73]:
# now break it down by type of perturbation
out = bound_df.groupby(["degree", "terms", "perturbation"]).agg(aggregations).reset_index()
out.to_csv(os.path.join(out_fldr, "bound_table_by_perturbation.csv"), index=False, mode="w")
out

/Users/sean/miniconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,degree,terms,perturbation,Disjunction (New),Disjunction (Old),VPCs (New),VPCs (Farkas),VPCs (All),VPCs (Disjunction),VPCs (Matrix),...,feasibleToInfeasibleTermsRatio Disjunction,feasibleToInfeasibleTermsRatio Matrix,feasibleToInfeasibleTermsRatio Term,feasibleToInfeasibleTermsRatio Basis,feasibleToInfeasibleTermsRatio NoDisjunction,feasibleToInfeasibleTermsRatio NoMatrix,feasibleToInfeasibleTermsRatio NoTerm,feasibleToInfeasibleTermsRatio NoBasis,instance,instanceIndex
0,1,4,matrix,0.139849,0.145539,0.139849,0.142625,0.142776,0.142625,0.142776,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1,20
1,1,4,objective,0.146033,0.146033,0.146033,0.146033,0.146033,0.146033,0.146033,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1,20
2,1,4,rhs,0.135827,0.153379,0.135204,0.144141,0.144141,0.144141,0.144141,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1,20
3,1,64,matrix,0.683543,0.707459,0.665729,0.579877,0.631318,0.615992,0.604382,...,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,1,19
4,1,64,objective,0.665545,0.707802,0.648594,0.686673,0.686673,0.686673,0.686673,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1,20
5,1,64,rhs,0.697293,0.707321,0.683371,0.221915,0.677023,0.675834,0.221915,...,0.000056,0.000056,0.000056,0.000056,0.000056,0.000056,0.000056,0.000056,1,12
6,4,4,matrix,0.101753,0.131983,0.100893,0.081293,0.106711,0.081293,0.106109,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1,20
7,4,4,objective,0.135393,0.135393,0.134473,0.135029,0.135029,0.135029,0.135029,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1,20
8,4,4,rhs,0.100090,0.097132,0.094838,0.044148,0.045303,0.044148,0.044148,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1,20
9,4,64,matrix,0.650560,0.593976,0.637210,0.000047,0.386260,0.006490,0.278970,...,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,1,12


In [76]:
# example table for VPC strength
out.loc[out["degree"] != 16, ["degree", "terms", "perturbation"] + [c for c in out.columns if "VPCs" in c and "No" not in c]]

,degree,terms,perturbation,VPCs (New),VPCs (Farkas),VPCs (All),VPCs (Disjunction),VPCs (Matrix),VPCs (Term),VPCs (Basis)
0,1,4,matrix,0.139849,0.142625,0.142776,0.142625,0.142776,0.142625,0.142699
1,1,4,objective,0.146033,0.146033,0.146033,0.146033,0.146033,0.146033,0.146033
2,1,4,rhs,0.135204,0.144141,0.144141,0.144141,0.144141,0.144141,0.144141
3,1,64,matrix,0.665729,0.579877,0.631318,0.615992,0.604382,0.579877,0.602940
4,1,64,objective,0.648594,0.686673,0.686673,0.686673,0.686673,0.686673,0.686673
5,1,64,rhs,0.683371,0.221915,0.677023,0.675834,0.221915,0.675834,0.222055
6,4,4,matrix,0.100893,0.081293,0.106711,0.081293,0.106109,0.081293,0.097366
7,4,4,objective,0.134473,0.135029,0.135029,0.135029,0.135029,0.135029,0.135029
8,4,4,rhs,0.094838,0.044148,0.045303,0.044148,0.044148,0.044148,0.045303
9,4,64,matrix,0.637210,0.000047,0.386260,0.006490,0.278970,0.004374,0.001140


In [78]:
# example table for root cut strength
out.loc[out["degree"] != 16, ["degree", "terms", "perturbation"] + [c for c in out.columns if "Root Cuts" in c and ("No" not in c or "None" in c)]]

,degree,terms,perturbation,Root Cuts (None),Root Cuts (New),Root Cuts (Farkas),Root Cuts (All),Root Cuts (Disjunction),Root Cuts (Matrix),Root Cuts (Term),Root Cuts (Basis)
0,1,4,matrix,0.398920,0.404974,0.411527,0.403333,0.407457,0.410932,0.415609,0.410111
1,1,4,objective,0.392745,0.405275,0.406176,0.406176,0.411752,0.406646,0.406646,0.411752
2,1,4,rhs,0.426073,0.425040,0.431691,0.426054,0.429141,0.429141,0.427994,0.419232
3,1,64,matrix,0.398958,0.682765,0.627764,0.661151,0.647471,0.649524,0.627920,0.647284
4,1,64,objective,0.393265,0.671348,0.699218,0.698985,0.698904,0.699257,0.699245,0.699230
5,1,64,rhs,0.397838,0.702378,0.663698,0.689303,0.687533,0.657862,0.688022,0.663982
6,4,4,matrix,0.427812,0.437714,0.427477,0.430341,0.419257,0.429140,0.429846,0.427865
7,4,4,objective,0.388715,0.387200,0.391059,0.391059,0.403393,0.403393,0.405134,0.405134
8,4,4,rhs,0.479165,0.488516,0.469783,0.474616,0.477070,0.472766,0.476564,0.469295
9,4,64,matrix,0.396124,0.656227,0.394649,0.499228,0.402980,0.452868,0.388891,0.412678


In [79]:
# example table for root cut generation time
out.loc[out["degree"] != 16, ["degree", "terms", "perturbation"] + [c for c in out.columns if "rootDualBoundTime" in c and ("No" not in c or "None" in c)]]

,degree,terms,perturbation,rootDualBoundTime None,rootDualBoundTime New,rootDualBoundTime Farkas,rootDualBoundTime All,rootDualBoundTime Disjunction,rootDualBoundTime Matrix,rootDualBoundTime Term,rootDualBoundTime Basis
0,1,4,matrix,0.107655,0.144390,0.119222,0.124247,0.113909,0.128455,0.118802,0.114579
1,1,4,objective,0.102651,0.148024,0.117392,0.117926,0.121894,0.118221,0.116372,0.121361
2,1,4,rhs,0.124094,0.152504,0.115794,0.112613,0.114080,0.112135,0.115603,0.114288
3,1,64,matrix,0.108258,0.547037,0.116554,0.170443,0.108435,0.248579,0.112675,0.121363
4,1,64,objective,0.101673,0.548731,0.108243,0.108996,0.108700,0.107993,0.108258,0.107539
5,1,64,rhs,0.120713,0.542864,0.110716,0.119164,0.102824,0.111143,0.108480,0.138374
6,4,4,matrix,0.116489,0.158355,0.120460,0.132958,0.115226,0.132313,0.116806,0.128283
7,4,4,objective,0.113976,0.162586,0.117013,0.115143,0.130566,0.128284,0.131415,0.126001
8,4,4,rhs,0.106285,0.147144,0.118419,0.114255,0.119928,0.115577,0.122824,0.119416
9,4,64,matrix,0.117297,0.551560,0.168027,0.197823,0.160558,0.296829,0.169707,0.251910


In [74]:
short_fields = ["Disjunction (New)"] + [f"VPCs ({gen_name})" for gen_name in ["New", "Farkas", "All"]] + \
    [f"Root Cuts ({gen_name})" for gen_name in ["None", "New", "Farkas", "All"]]
aggregations = {f: geometric_mean for f in short_fields}
full_df[(full_df["terms"] == 64) & (full_df["perturbation"] == "matrix") & (full_df["degree"] == 1)].groupby("instance").agg(aggregations)

,Disjunction (New),VPCs (New),VPCs (Farkas),VPCs (All),Root Cuts (None),Root Cuts (New),Root Cuts (Farkas),Root Cuts (All)
instance,,,,,,,,
aflow30a,0.287611,0.056694,0.000003,0.000003,0.813051,0.807032,0.812087,0.816214
bienst1,0.467405,0.070822,0.019365,0.019077,0.239700,0.303949,0.307944,0.298613
binkar10_1,0.099609,0.042558,0.042558,0.042558,0.736321,0.757084,0.734906,0.751016
blp-ir98,0.071322,0.035797,0.027323,0.035797,0.832842,0.839068,0.809065,0.867985
bppc8-09,0.505930,0.008232,0.000001,0.000001,0.599779,0.600207,0.599784,0.600199
cap6000,0.414146,0.106891,0.000534,0.000534,0.540083,0.548868,0.548868,0.548868
exp-1-500-5-5,0.105227,0.007480,0.000012,0.000012,0.904398,0.884379,0.891576,0.897992
g200x740,0.009000,0.007624,0.000004,0.000004,0.929728,0.933107,0.930347,0.932143
graphdraw-gemcutter,0.000031,0.000004,0.000001,0.000001,0.082125,0.085770,0.081114,0.081114


## Check Termination Stats

In [75]:
# additional filtering for dataframe on run time
fields = [f"terminationTime {gen}" for gen in generators] + \
         [f"terminationTimeImprovement {gen}" for gen in generators if gen not in ["None", "New"]]
# only check perturbed instances that solve to optimality and VPC didn't find optimal solution
mask = (df["Disjunction (New)"] < .9999) & (df["instanceIndex"] > 0) & (optimality_gap(df, "New") <= 1e-4) & \
    (optimality_gap(df, "None") <= 1e-4) & (optimality_gap(df, "Farkas") <= 1e-4) & \
       (df["terminationTime None"] > min_termination_time)

# create time dataframe
time_df = df.loc[mask, group_cols + id_cols + fields]

In [79]:
aggregations = {f"Average Time {gen}": (f"terminationTime {gen}", geometric_mean) for gen in generators} | \
    {f"Average Improvement {gen}": (f"terminationTimeImprovement {gen}", "mean") for gen in generators if gen not in ["None", "New"]} | \
    {"count": ("terminationTimeImprovement Farkas", "size")}

tmp = time_df.groupby(["instance", "perturbation", "degree", "terms"]).agg(**aggregations).reset_index()
tmp = tmp[(tmp["count"] > 1)]
tmp.to_csv(os.path.join(out_fldr, "high_perform_all.csv"), index=False, mode="w")
tmp.head()

,instance,perturbation,degree,terms,Average Time None,Average Time New,Average Time Farkas,Average Time All,Average Time Disjunction,Average Time Matrix,...,Average Improvement All,Average Improvement Disjunction,Average Improvement Matrix,Average Improvement Term,Average Improvement Basis,Average Improvement NoDisjunction,Average Improvement NoMatrix,Average Improvement NoTerm,Average Improvement NoBasis,count
0,aflow30a,matrix,1,4,3.397194,6.908022,3.193351,2.461512,3.536989,3.335433,...,0.244497,-0.192570,-0.114798,0.221280,0.223497,0.247561,-0.445772,-0.112080,0.009604,9
1,aflow30a,matrix,1,64,5.262113,8.670610,5.423080,5.206116,4.875512,4.878356,...,-0.123315,-0.052529,-0.051905,-0.029195,-0.028607,-0.149072,-0.032196,-0.218607,-0.106109,8
2,aflow30a,rhs,1,4,2.071641,8.422406,2.870855,2.752171,2.240980,2.532260,...,-0.489788,-0.205667,-0.278145,-0.124080,-0.288904,-0.201035,0.035995,-0.733652,-0.384405,3
3,aflow30a,rhs,1,64,3.124030,7.635770,3.190507,3.526027,3.366901,2.722832,...,-0.207908,-0.272866,0.085129,0.172113,-0.470320,-0.011076,-0.096411,-0.015672,-0.195049,3
4,aligninq,matrix,1,4,129.830683,130.857454,123.017751,113.325551,126.388712,126.584325,...,-0.018756,-0.076447,-0.078747,0.158329,-0.087549,-0.199517,0.054263,-0.076295,0.054687,10


In [127]:
def make_improvement_table(tmp, generator):
    
    # columns we always choose
    key_cols = ["degree", "terms", "perturbation", "instance"]
    time_cols = [f"Average Time {g}" for g in ["None", "New", "Farkas"]]
    
    # subset the ones we want
    all_df = tmp[
        key_cols + time_cols + [f"Average Time {generator}", f"Average Improvement Farkas", f"Average Improvement {generator}", "count"]
    ].sort_values(f"Average Improvement {generator}", ascending=False)
    all_df = all_df[all_df[f"Average Improvement {generator}"] > 0]
    best_df = all_df.loc[
        all_df.groupby(['perturbation', 'degree', 'terms'])[f'Average Improvement {generator}'].idxmax()
    ].sort_values(f"Average Improvement {generator}", ascending=False)
    
    # save all the winners
    all_df.to_csv(os.path.join(out_fldr, f"high_perform_{generator.lower()}.csv"), index=False, mode="w")
    
    # return just the best
    return all_df, best_df

In [137]:
all_df, best_df = make_improvement_table(tmp, "Disjunction")
best_df.loc[[285, 392, 213, 373, 186, 172], [c for c in best_df.columns if "Improvement" not in c]]

,degree,terms,perturbation,instance,Average Time None,Average Time New,Average Time Farkas,Average Time Disjunction,count
285,1,4,matrix,neos-860300,129.235105,97.853468,70.466980,40.607542,10
392,1,4,objective,ran13x13,35.537751,14.976852,17.016891,13.095101,10
213,4,4,rhs,neos-3046615-murg,231.783260,168.801212,200.152352,150.464393,2
373,4,64,rhs,pg5_34,3.150931,71.660436,2.063292,1.944108,8
186,16,4,matrix,n7-3,8.888611,30.491732,13.156323,5.293075,2
172,16,64,objective,misc07,83.671824,47.174409,41.371435,28.676896,10


In [138]:
all_df, best_df = make_improvement_table(tmp, "Matrix")
best_df.loc[best_df["perturbation"] == "matrix", [c for c in best_df.columns if "Improvement" not in c]].sort_values(["degree", "terms"])

,degree,terms,perturbation,instance,Average Time None,Average Time New,Average Time Farkas,Average Time Matrix,count
285,1,4,matrix,neos-860300,129.235105,97.853468,70.466980,50.045030,10
388,1,64,matrix,ran13x13,22.612178,47.155872,20.394313,15.924265,10
332,4,4,matrix,neos18,5.268503,11.776036,4.612012,3.310361,9
75,4,64,matrix,g200x740,26.799084,62.360644,33.217829,20.611519,2
186,16,4,matrix,n7-3,8.888611,30.491732,13.156323,5.340057,2
296,16,64,matrix,neos-911970,29.294123,26.230940,26.757343,19.561260,6


In [148]:
all_df, best_df = make_improvement_table(tmp, "Term")
best_df = best_df.loc[(best_df["perturbation"] == "matrix") | (best_df["perturbation"] == "rhs"),
                      [c for c in best_df.columns if "Improvement" not in c]]
best_df.loc[[285, 373, 410, 296, 131, 341]].sort_values(["degree", "terms"])

,degree,terms,perturbation,instance,Average Time None,Average Time New,Average Time Farkas,Average Time Term,count
285,1,4,matrix,neos-860300,129.235105,97.853468,70.466980,39.316548,10
341,1,64,rhs,neos18,13.655170,638.616928,16.640351,9.637685,2
410,4,4,matrix,rout,10.085790,8.247897,10.493955,6.188331,10
373,4,64,rhs,pg5_34,3.150931,71.660436,2.063292,1.975149,8
131,16,4,rhs,mas76,47.049209,63.572653,44.410446,38.407041,4
296,16,64,matrix,neos-911970,29.294123,26.230940,26.757343,19.431839,6


In [149]:
all_df, best_df = make_improvement_table(tmp, "Basis")
best_df = best_df.loc[(best_df["perturbation"] == "matrix") | (best_df["perturbation"] == "rhs"),
                      [c for c in best_df.columns if "Improvement" not in c]]
best_df.loc[[285, 341, 195, 390, 296, 295]].sort_values(["degree", "terms"])

,degree,terms,perturbation,instance,Average Time None,Average Time New,Average Time Farkas,Average Time Basis,count
285,1,4,matrix,neos-860300,129.235105,97.853468,70.466980,37.981369,10
341,1,64,rhs,neos18,13.655170,638.616928,16.640351,9.832116,2
195,4,4,rhs,neos-1445743,43.887964,2706.963349,36.087721,24.296440,4
390,4,64,matrix,ran13x13,54.497417,66.367305,77.418971,42.131783,10
295,16,4,matrix,neos-911970,19.315416,18.745702,17.694233,10.563624,5
296,16,64,matrix,neos-911970,29.294123,26.230940,26.757343,16.078533,6


In [150]:
all_df, best_df = make_improvement_table(tmp, "All")
best_df.loc[[285, 120, 35, 390, 301, 172], [c for c in best_df.columns if "Improvement" not in c]].sort_values(["degree", "terms"])

,degree,terms,perturbation,instance,Average Time None,Average Time New,Average Time Farkas,Average Time All,count
285,1,4,matrix,neos-860300,129.235105,97.853468,70.466980,56.302859,10
120,1,64,matrix,mas76,17.747258,38.971226,13.788161,9.943748,4
35,4,4,rhs,blp-ir98,12.691349,143.314590,8.665618,7.336168,2
390,4,64,matrix,ran13x13,54.497417,66.367305,77.418971,45.969478,10
301,16,4,objective,neos-911970,102.516404,69.198447,139.244088,41.855894,2
172,16,64,objective,misc07,83.671824,47.174409,41.371435,35.810067,10


In [93]:
# find ratios of all vs farkas
# ijoc santanu and prachi paper on adding one cut and tree blows up
# are the cuts getting stronger?
# does time (excluding cut generation) improve when tightening improves